# Imports

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Data Clean

In [9]:
dataset_names = {'pCO2': 'pCO2_2D_mon_CESM001_1x1_198201-201701.nc',
                 'XCO2': 'XCO2_1D_mon_CESM001_native_198201-201701.nc',
                 'SST': 'SST_2D_mon_CESM001_1x1_198201-201701.nc',
                 'SSS': 'SSS_2D_mon_CESM001_1x1_198201-201701.nc',
                 'MLD': 'MLD_2D_mon_CESM001_1x1_198201-201701.nc',
                 'Chl': 'Chl_2D_mon_CESM001_1x1_198201-201701.nc'}
ds = {}
for dataset in dataset_names.keys():
    filename = os.path.join(dataset_names[dataset])
    ds[dataset] = xr.open_dataset(filename)

In [10]:
merged_dataset = xr.merge([ds[name][name] for name in ds.keys()])
merged_dataset = xr.merge([merged_dataset, ds['pCO2']])

In [11]:
df = merged_dataset.to_dataframe().reset_index()

In [12]:
df.dropna(subset=['time','xlon', 'ylat','pCO2', 'XCO2', 'SST', 'SSS', 'MLD', 'Chl'], inplace=True)

In [13]:
df = df[['time','xlon', 'ylat','pCO2', 'XCO2', 'SST', 'SSS', 'MLD', 'Chl']].reset_index()

In [14]:
sample = pd.read_csv('SOM_all_data.csv')

In [15]:
df['s0'] = np.sin(df.ylat)
df['s1'] = np.sin(df.xlon) * np.cos(df.ylat)
df['s2'] = -np.cos(df.xlon) * np.cos(df.ylat)
df['t0'] = np.cos(df.time.dt.dayofyear * 2 * np.pi / 365)
df['t1'] = np.sin(df.time.dt.dayofyear * 2 * np.pi / 365)
df['month'] = df.time.dt.month
df['year'] = df.time.dt.year

In [16]:
merged = pd.merge(sample, df, how='left', on=['xlon', 'ylat', 'month', 'year'])

In [17]:
merged = merged[['pCO2_x','XCO2', 'SST_x', 'SSS_x', 'MLD_x', 'Chl','s0','s1','s2','t0','t1','k=10']]

In [18]:
merged.head()

,pCO2_x,XCO2,SST_x,SSS_x,MLD_x,Chl,s0,s1,s2,t0,t1,k=10
0,352.818516,354.673737,27.900078,34.516457,34.473396,0.110500,-0.591358,0.672732,-0.444666,-0.251190,-0.967938,2
1,312.662032,346.094177,16.368843,34.548840,83.076450,0.227699,0.809019,-0.196408,-0.553997,-0.969010,-0.247022,1
2,331.637467,383.021515,2.618722,33.257133,23.466050,0.119260,-0.455956,0.160570,0.875398,0.702527,0.711657,1
3,328.017932,354.964233,3.973839,32.190420,59.333320,0.199080,0.998591,0.052602,0.007025,0.966848,-0.255353,2
4,339.139662,354.186523,13.662021,34.104397,53.165960,0.208976,0.996087,0.020538,-0.085964,-0.251190,0.967938,1


In [20]:
merged =pd.get_dummies(merged, columns = ['k=10'], 
                         prefix = ['cluster'], 
                         dummy_na = False)

In [21]:
merged.head()

,pCO2_x,XCO2,SST_x,SSS_x,MLD_x,Chl,s0,s1,s2,t0,...,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9
0,352.818516,354.673737,27.900078,34.516457,34.473396,0.110500,-0.591358,0.672732,-0.444666,-0.251190,...,0,0,1,0,0,0,0,0,0,0
1,312.662032,346.094177,16.368843,34.548840,83.076450,0.227699,0.809019,-0.196408,-0.553997,-0.969010,...,0,1,0,0,0,0,0,0,0,0
2,331.637467,383.021515,2.618722,33.257133,23.466050,0.119260,-0.455956,0.160570,0.875398,0.702527,...,0,1,0,0,0,0,0,0,0,0
3,328.017932,354.964233,3.973839,32.190420,59.333320,0.199080,0.998591,0.052602,0.007025,0.966848,...,0,0,1,0,0,0,0,0,0,0
4,339.139662,354.186523,13.662021,34.104397,53.165960,0.208976,0.996087,0.020538,-0.085964,-0.251190,...,0,1,0,0,0,0,0,0,0,0


In [22]:
X = np.array(merged.iloc[:,1:])

In [23]:
Y = np.array(merged.iloc[:,:1])
Y = Y.ravel()

# Models

# Decision Tree

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.25, random_state = 42)

In [59]:
parameters = {'max_depth':[15,17,20,25,27,30],'min_samples_split':[20,25,30,35,40] }
regressor = DecisionTreeRegressor(random_state=0, criterion='mse')
reg = GridSearchCV(regressor, parameters, cv=3, n_jobs=-1)
#regressor = DecisionTreeRegressor(max_depth = 23, min_samples_split=35, random_state=42, criterion='mse')

In [27]:
reg = DecisionTreeRegressor(random_state=0, criterion='mse', 
                            max_depth = 25, min_samples_split = 30)

In [28]:
reg.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=30, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [29]:
#print('Best Parameters for Tree are:', reg.best_params_)
print('Tree - Train R-squared', reg.score(X_train, y_train))
print('Tree - Test R-squared', reg.score(X_test, y_test))

Tree - Train R-squared 0.9707675767565365
Tree - Test R-squared 0.9580099872175026


In [30]:
np.sqrt(mean_squared_error(y_train, reg.predict(X_train)))

7.62026753553316

In [31]:
np.sqrt(mean_squared_error(y_test, reg.predict(X_test)))

9.128915167846191

In [35]:
np.argsort(reg.feature_importances_)[::-1]

array([19, 18,  0,  4,  1,  2,  3,  9, 12,  8, 16, 14,  5, 10, 11, 17, 13,
        6,  7, 15])

In [40]:
reg.feature_importances_

array([1.30612404e-01, 1.00952182e-01, 6.04215572e-02, 5.94394260e-02,
       1.12295933e-01, 6.96309189e-03, 4.84108846e-04, 3.89936910e-04,
       2.10760900e-02, 3.84064189e-02, 3.43959636e-03, 2.47464985e-03,
       2.51105873e-02, 1.50324014e-03, 7.00267651e-03, 5.00796889e-06,
       2.01098164e-02, 2.11241744e-03, 1.61145647e-01, 2.46055212e-01])

In [36]:
merged.iloc[:,1:].columns[np.argsort(reg.feature_importances_)[::-1]]

Index(['cluster_9', 'cluster_8', 'XCO2', 'Chl', 'SST_x', 'SSS_x', 'MLD_x',
       't1', 'cluster_2', 't0', 'cluster_6', 'cluster_4', 's0', 'cluster_0',
       'cluster_1', 'cluster_7', 'cluster_3', 's1', 's2', 'cluster_5'],
      dtype='object')

# Random Forest

In [29]:
parameters = {'max_depth':[5,7,9,10,12,15],'min_samples_split':[5,10,15,20,30] }
regr = RandomForestRegressor(random_state=0, n_estimators= 51, n_jobs=-1)
reg = GridSearchCV(regr, parameters, cv=3)

In [30]:
y_train = y_train.ravel()

In [31]:
reg.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print('Best Parameters for Random Forest are:', reg.best_params_)
print('Random Forest - Train R-squared', reg.score(X_train, y_train))
print('Random Forest - Test R-squared', reg.score(X_test, y_test))

In [ ]:
np.sqrt(mean_squared_error(y_train, reg.predict(X_train)))

In [ ]:
np.sqrt(mean_squared_error(y_test, reg.predict(X_test)))